# Importing the Data

In [1]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.pipeline import make_pipeline 

# import datetime
import datetime

# encoders
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# model creation
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV 
import pickle

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', 100)

In [2]:
# load the csvs - it is available open source and online
df = pd.read_csv('2019_Winter_Data_Science_Intern_Challenge_Data_Set.csv')
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [3]:
# order id can be dropped since theres only unique values
df['order_id'].value_counts().value_counts()

1    5000
Name: order_id, dtype: int64

# App Question

Question 1: Given some sample data, write a program to answer the following

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
What metric would you report for this dataset?
What is its value?


# Data Exploration

In [4]:
def wrangle(path):
    df = pd.read_csv(path)

    # convert created_at column to a datetime object
    df['created_at']= pd.to_datetime(df['created_at'])

    # sort the dataframe by date
    df = df.sort_values(by=['created_at'])

    # drop high-cardinality and irrelevant features
    df.drop('order_id', axis=1, inplace=True)

    return df

In [5]:
path = '2019_Winter_Data_Science_Intern_Challenge_Data_Set.csv'
df = wrangle(path)
df.head()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
1862,39,738,536,4,cash,2017-03-01 00:08:09
1741,39,910,268,2,cash,2017-03-01 00:10:19
3228,97,912,324,2,cash,2017-03-01 00:14:12
1267,80,798,290,2,credit_card,2017-03-01 00:19:31
2689,49,799,258,2,credit_card,2017-03-01 00:22:25


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1862 to 2457
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   shop_id         5000 non-null   int64         
 1   user_id         5000 non-null   int64         
 2   order_amount    5000 non-null   int64         
 3   total_items     5000 non-null   int64         
 4   payment_method  5000 non-null   object        
 5   created_at      5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 273.4+ KB


In [7]:
df.tail()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
2630,53,940,112,1,credit_card,2017-03-30 23:12:13
1685,34,818,244,2,cash,2017-03-30 23:16:10
1474,21,815,142,1,cash,2017-03-30 23:26:54
317,52,848,292,2,cash,2017-03-30 23:41:34
2457,95,700,168,1,credit_card,2017-03-30 23:55:35


In [8]:
df['payment_method'].value_counts()

credit_card    1735
debit          1671
cash           1594
Name: payment_method, dtype: int64

In [9]:
df.sort_values(by=['order_amount'])

,shop_id,user_id,order_amount,total_items,payment_method,created_at
4760,92,937,90,1,debit,2017-03-20 07:37:28
1843,92,987,90,1,debit,2017-03-06 07:01:09
2092,92,986,90,1,debit,2017-03-04 06:44:05
4414,92,927,90,1,credit_card,2017-03-17 09:57:01
228,92,757,90,1,debit,2017-03-13 23:57:51
...,...,...,...,...,...,...
4868,42,607,704000,2000,credit_card,2017-03-22 04:00:00
15,42,607,704000,2000,credit_card,2017-03-07 04:00:00
1602,42,607,704000,2000,credit_card,2017-03-17 04:00:00
2969,42,607,704000,2000,credit_card,2017-03-28 04:00:00


In [10]:
# any order amount greater than $10000 came from two different stores
df[df['order_amount'] > 10000]

,shop_id,user_id,order_amount,total_items,payment_method,created_at
4311,78,960,51450,2,debit,2017-03-01 03:02:10
4646,42,607,704000,2000,credit_card,2017-03-02 04:00:00
520,42,607,704000,2000,credit_card,2017-03-02 04:00:00
4412,78,756,51450,2,debit,2017-03-02 04:13:39
4040,78,852,25725,1,cash,2017-03-02 14:31:12
2821,78,814,51450,2,cash,2017-03-02 17:13:25
60,42,607,704000,2000,credit_card,2017-03-04 04:00:00
2492,78,834,102900,4,debit,2017-03-04 04:37:34
4715,78,818,77175,3,debit,2017-03-05 05:10:44
2297,42,607,704000,2000,credit_card,2017-03-07 04:00:00


# Using Interquartile Range to Find Outliers

In [11]:
df.describe()

,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.00000
mean,50.078800,849.092400,3145.128000,8.78720
std,29.006118,87.798982,41282.539349,116.32032
min,1.000000,607.000000,90.000000,1.00000
25%,24.000000,775.000000,163.000000,1.00000
50%,50.000000,849.000000,284.000000,2.00000
75%,75.000000,925.000000,390.000000,3.00000
max,100.000000,999.000000,704000.000000,2000.00000


In [12]:
# Use the 25% and 75& range of the order_amount to calculate outlier
Q1_Order_Amount_Outlier = 163
Q3_Order_Amount_Outlier = 390
IQR_Order_Amount = Q3_Order_Amount_Outlier - Q1_Order_Amount_Outlier

# Calculate the outlier
Minimum_Outlier = Q1_Order_Amount_Outlier - (1.5*IQR_Order_Amount)
Maximum_Outlier = Q3_Order_Amount_Outlier + (1.5*IQR_Order_Amount)

print('Minimum Outlier: ', Minimum_Outlier)
print('Maximum Outlier: ', Maximum_Outlier)

Minimum Outlier:  -177.5
Maximum Outlier:  730.5


In [13]:
# Only need to use Maximum outlier as a condition since minimum outlier is negative
Outlier_Condition = (df['order_amount'] <= Maximum_Outlier)

# create a df without the outliers
df_no_outlier = df[Outlier_Condition]

# sanity check to see if outliers removed
df_no_outlier.sort_values(by=['order_amount']).tail()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
3824,43,736,724,4,credit_card,2017-03-25 17:34:51
2786,43,876,724,4,credit_card,2017-03-01 01:41:43
2312,79,745,724,4,debit,2017-03-27 09:26:31
4659,47,894,725,5,cash,2017-03-01 04:49:59
1124,52,994,730,5,credit_card,2017-03-07 06:54:05


In [14]:
df_no_outlier.describe()

,shop_id,user_id,order_amount,total_items
count,4859.000000,4859.000000,4859.000000,4859.000000
mean,49.852645,849.905742,293.715374,1.950196
std,29.049171,86.887496,144.453395,0.919791
min,1.000000,700.000000,90.000000,1.000000
25%,24.000000,776.000000,162.000000,1.000000
50%,50.000000,850.000000,280.000000,2.000000
75%,74.000000,925.000000,380.000000,3.000000
max,100.000000,999.000000,730.000000,5.000000


# Average Order Value of Each Store

In [15]:
# Total number of shops
print('Total number of shops: ', len(df['shop_id'].value_counts()))

Total number of shops:  100


In [16]:
# sort df by shop number for iteration
df = df.sort_values(by=['shop_id'])

# reset index for iteration
df.reset_index(drop=True, inplace=True)

df.head()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,705,474,3,debit,2017-03-13 03:13:36
1,1,934,316,2,debit,2017-03-07 03:29:47
2,1,924,474,3,credit_card,2017-03-16 02:31:40
3,1,951,158,1,credit_card,2017-03-04 20:02:50
4,1,877,474,3,cash,2017-03-09 17:31:51


In [20]:
# empty lists used for iterative purpose
order_amounts = []
item_counts = []

# lists to be used to create new df
shops = []
AOV = []
avg_num_item_sold = []
sum_sales = []
sum_items = []

# iterate shop id to find AOV and average item_counts
for i, val in enumerate(df['shop_id']):
    if val not in shops:
        shops.append(val)
        # if list is not empty (meaning its not the very 1st iteration)
        if order_amounts:
            # append average and sums to average and sum lists
            AOV.append(sum(order_amounts) / len(order_amounts))
            avg_num_item_sold.append(sum(item_counts) / len(item_counts))
            sum_sales.append(sum(order_amounts))
            sum_items.append(sum(item_counts))
            # clear iterative lists for next shop id
            order_amounts.clear()
            item_counts.clear()
            # add the 1st order amount and item count to next shop id iterative list
            order_amounts.append(df['order_amount'][i])
            item_counts.append(df['total_items'][i])
        else:
            # if list is empty (first iteration) it will add first order amount and item count
            order_amounts.append(df['order_amount'][i])
            item_counts.append(df['total_items'][i])
    else:
        # continue to add order amounts and item counts for the same shop id
        order_amounts.append(df['order_amount'][i])
        item_counts.append(df['total_items'][i])

# append values for the final iteration
AOV.append(sum(order_amounts) / len(order_amounts))
avg_num_item_sold.append(sum(item_counts) / len(item_counts))
sum_sales.append(sum(order_amounts))
sum_items.append(sum(item_counts))

In [26]:
# assert that iteration worked correctly
assert (len(AOV) == len(avg_num_item_sold)) & (len(AOV) == len(sum_items))
assert (len(AOV) == len(sum_sales))

In [29]:
# round numbers in lists
AOV_rounded = [round(num, 2) for num in AOV]
avg_num_item_sold_rounded = [round(num, 2) for num in avg_num_item_sold]

In [35]:
# create store_id list
store_id = [num for num in range(1,101)]
len(store_id)

100

In [46]:
# create dictionary before converting to df
store_data = {'AOV': AOV_rounded, 'avg_num_item_sold': avg_num_item_sold_rounded, 
              'sum_sales': sum_sales, 'sum_items': sum_items}

# create df
df_store = pd.DataFrame(data=store_data, index=store_id)
df_store.index.name = 'store_id'

In [47]:
df_store

,AOV,avg_num_item_sold,sum_sales,sum_items
store_id,,,,
1,308.82,1.95,13588,86
2,174.33,1.85,9588,102
3,305.25,2.06,14652,99
4,258.51,2.02,13184,103
5,290.31,2.04,13064,92
6,383.51,2.05,22627,121
7,218.00,1.95,12208,109
8,241.04,1.83,11088,84
9,234.00,1.98,13806,117


In [ ]:
        # add a column to df calculating the avg order amount
        order_amounts.append(df['order_amount'][i])
        df['AOV_per_store'][i] = sum(order_amounts) / len(order_amounts)
        # add a column to df calculating the avg total_items sold
        item_counts.append(df['total_items'][i])
        df['avg_num_sold_items'][i] = sum(item_counts) / len(item_counts)


        # add a column to df calculating the avg order amount
        order_amounts.append(df['order_amount'][i])
        df['AOV_per_store'][i] = sum(order_amounts) / len(order_amounts)
        # add a column to df calculating the avg total_items sold
        item_counts.append(df['total_items'][i])
        df['avg_num_sold_items'][i] = sum(item_counts) / len(item_counts)

In [18]:
df.head(100)

,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,705,474,3,debit,2017-03-13 03:13:36
1,1,934,316,2,debit,2017-03-07 03:29:47
2,1,924,474,3,credit_card,2017-03-16 02:31:40
3,1,951,158,1,credit_card,2017-03-04 20:02:50
4,1,877,474,3,cash,2017-03-09 17:31:51
5,1,725,316,2,credit_card,2017-03-09 06:58:46
6,1,788,316,2,credit_card,2017-03-18 12:55:40
7,1,893,316,2,credit_card,2017-03-11 19:55:28
8,1,786,474,3,credit_card,2017-03-09 06:24:43
9,1,715,316,2,debit,2017-03-08 10:33:04


# Answer

At first glance, it seems the reason that the average (mean) order amount is so high is that there are some values are outliers (there an order that ordered 2,000 itmes for a total amount of $704,000).

One Solution is to find the median instead of the mean so that it's easier to ignore the outliers. I found the median to be $284, which is a much more believable amount.

Another solution is to find the outlier for the order amount, and create a new dataset with the outliers removed since the outliers are outrageous in amount and are either a mistake or a huge unique type transaction (i.e. warehouse inventory order). After removing the outliers, we found the average (mean) order amount to be ~$294. To note, the median didn't change much with median order amount being $280. Therefore I'm pretty confident using the $294 as the average order amount.

However, since each store only sells one product, it might make sense to calculate the average order value for each store individually.